# Formula Recalculation for Mineral Analyses

This code is the procedure for the recalculation of mineral formulas using as input the oxide content of mineral anlyses, i.e. mass% of oxides from EDS or WDS analyses

## 1. Divide oxide mass% input by oxide molecular weights
   #### mass% / mol wt = mol amount 
      Ex.: measured SiO2 in a biotite crystal = 52.8 mass%; molar weight of SiO2 = 60.0843
      52.8 / 60.0843 = 0.878765

## 2. Multiply mol amount by amount of oxygen in the oxide
   #### mol amount * oxygen in oxide = oxygen number
       Ex.: molar amount in our case = 0.878765; SiO2 has 2 oxygens in the oxide
       0.878765 * 2 = 1.757531
       
## 3. Sum up the calculated oxygen number for all oxides
An exception here is the occurrence of F or Cl in the analyses. As these commonly occur as anions, they should be subtracted from the total sum. Both elements have -1 charge, which is half the charge of oxygen anions. Consequently, their calculated oxygen number should be multiplied by 0.5 before subtraction of the total.
   #### Σ oxygen numbers of all cations - Σ (0.5 * oxygen numbers of F and Cl) = total oxygen
       Ex.: oxygen number for SiO2 in our case = 1.757531; 
       summing up with other oxides is = 4.58;
       oxygen number for F and Cl is = 0.30 and 0.002 respectively 	
       4.58 - [(0.5 * 0.3)+(0.5 * 0.002)] = 4.429
       
## 4. Divide the number of oxygens in the mineral formula unit by total oxygen of last step
The oxygen in the formula unit can be retrieved from the ideal mineral formula. For anhydrous minerals, this value is directly the number of oxygens listed (e.g. 4 in the case of olivine, (Fe,Mg)SiO4). For hydrated phases each other anion (OH, F, or Cl) will count as half oxygen and the final value will be the number of oxygens plus half the number of OH,F,Cl in the formula (e.g. 11 for biotite K(Mg,Fe)3AlSi3O10(OH,F)2)
   #### O in formula unit / total oxygen = oxygen normalization factor 
       Ex.: total oxygen in our case = 4.429; biotite has an equivalent value of 11 oxygens in its formula (considering also the amount of OH and F in the ideal formula - see paragraph above for further clarification)
       11 / 4.429 = 2.484
       
## 5. Multiply the oxygen number from item 2 by the oxygen normalization factor 
   #### oxygen number * ONF = anionic proportion
       Ex.: oxygen for SiO2 in our case = 1.757531; ONF = 2.484
       1.757531 * 2.484 = 4.365
       
## 6. Multiply the anionic proportion by the ratio of cations and oxygen in the oxide 
   #### anionic proportion * (cations in oxide / oxygen in oxide) = cationic proportion
       Ex.: in our case anionic proportion = 4.365; and the ratio between Si and O in the oxide formula (i.e. SiO2) is 1/2 = 0.5
       4.365 * 0.5 = 2.183

In [1]:
    # --- import required modules

import numpy as np
import pandas as pd

In [2]:
    # --- create pandas dataframe from external file 
    
df_analysis = pd.read_csv("_DATA/Ore_WDS.csv")

    # --- print the columns of the imported dataframe 
    
df_analysis.columns 

Index(['Sample', 'Area', 'Point', 'Comment', 'mineral', 'Al2O3', 'Ta2O5',
       'TiO2', 'MnO', 'MgO', 'FeO', 'SnO2', 'CaO', 'Nb2O5', 'Total'],
      dtype='object')

In [3]:
 # --- print analysis data for initial assessment

df_analysis

,Sample,Area,Point,Comment,mineral,Al2O3,Ta2O5,TiO2,MnO,MgO,FeO,SnO2,CaO,Nb2O5,Total
0,BU04,Ore1,2,BU04-Ore1.2,coltan,0.538319,7.363233,0.877254,11.932050,0.0,8.465170,0.094458,0.158389,71.010020,100.438893
1,BU04,Ore1,3,BU04-Ore1.3,coltan,0.032877,29.220923,1.287440,8.070750,0.0,9.893185,0.534629,0.027564,51.440780,100.508148
2,BU04,Ore1,4,BU04-Ore1.4,coltan,0.018895,25.948375,1.526645,10.033050,0.0,8.439440,0.532724,0.020988,54.087205,100.607322
3,BU04,Ore1,5,BU04-Ore1.5,coltan,0.056118,19.195692,1.564678,9.732375,0.0,9.172745,0.491716,0.076116,59.194090,99.483531
4,BU04,Ore1,6,BU04-Ore1.6,coltan,0.046482,16.912235,1.951677,9.684900,0.0,9.623020,0.507586,0.022247,59.709070,98.457217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,BU19w,Ore4,1,BU19w-Rut4.1,rutile,0.704000,6.410000,84.020000,0.037800,0.0,3.570000,0.573700,NaN,5.220000,100.535500
88,BU19w,Ore4,2,BU19w-Rut4.2,rutile,0.616000,4.520000,91.840000,0.009100,0.0,2.290000,0.047900,NaN,3.130000,102.453000
89,BU19w,Ore4,3,BU19w-Rut4.3,rutile,2.410000,9.220000,73.180000,0.024200,0.0,4.510000,1.066100,NaN,7.220000,97.630300
90,BU19w,Ore5,2,BU19w-Area7_ore.2,rutile,0.291200,6.600000,81.070000,0.036200,0.0,4.130000,1.400000,NaN,8.020000,101.547400


In [4]:
    # --- separate data from metadata - only do calculation in data afterwards
    
df_data = df_analysis.drop(['Sample','Area', 'Point', 'Comment', 'mineral','Total'], axis=1)

df_data

,Al2O3,Ta2O5,TiO2,MnO,MgO,FeO,SnO2,CaO,Nb2O5
0,0.538319,7.363233,0.877254,11.932050,0.0,8.465170,0.094458,0.158389,71.010020
1,0.032877,29.220923,1.287440,8.070750,0.0,9.893185,0.534629,0.027564,51.440780
2,0.018895,25.948375,1.526645,10.033050,0.0,8.439440,0.532724,0.020988,54.087205
3,0.056118,19.195692,1.564678,9.732375,0.0,9.172745,0.491716,0.076116,59.194090
4,0.046482,16.912235,1.951677,9.684900,0.0,9.623020,0.507586,0.022247,59.709070
...,...,...,...,...,...,...,...,...,...
87,0.704000,6.410000,84.020000,0.037800,0.0,3.570000,0.573700,NaN,5.220000
88,0.616000,4.520000,91.840000,0.009100,0.0,2.290000,0.047900,NaN,3.130000
89,2.410000,9.220000,73.180000,0.024200,0.0,4.510000,1.066100,NaN,7.220000
90,0.291200,6.600000,81.070000,0.036200,0.0,4.130000,1.400000,NaN,8.020000


In [5]:
len(df_data.columns)

9

In [6]:
    # --- create a reference dataframe with data of oxides

df_reference = pd.read_csv("_DATA/_Oxides_mass.csv",index_col=0)

reference_oxides = df_reference[df_data.columns]

reference_oxides

,Al2O3,Ta2O5,TiO2,MnO,MgO,FeO,SnO2,CaO,Nb2O5
Oxide,,,,,,,,,
Mol_wt,101.961280,441.892800,79.898800,70.937400,40.304400,71.846400,150.68880,56.079400,265.809800
Factor,0.529251,0.818967,0.599508,0.774457,0.603036,0.777311,0.78765,0.714701,0.699044
Ox,3.000000,5.000000,2.000000,1.000000,1.000000,1.000000,2.00000,1.000000,5.000000
Cat,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,2.000000
Atom_wt,26.980640,180.946400,47.898800,54.937400,24.304400,55.846400,118.68880,40.079400,92.904900


In [7]:
    # --- create an array with molecular weights of oxides
    
molecular_weights = reference_oxides.iloc[0,:]

    # --- create an array with the amount of oxygen in each of the oxides

oxygen_in_ox = reference_oxides.iloc[2,:]

    # --- create an array with the ratio between cations and oxygen in oxides
    
cation_ratio_ox = reference_oxides.iloc[3].div(reference_oxides.iloc[2])

print(len(molecular_weights),
      len(oxygen_in_ox),
      len(cation_ratio_ox))

9 9 9


## 1. Divide oxide mass% input by oxide molecular weights
   #### mass% / mol wt = mol amount 
      Ex.: measured SiO2 in a biotite crystal = 52.8 mass%; molar weight of SiO2 = 60.0843
      52.8 / 60.0843 = 0.878765

In [8]:
    # --- new dataframe with measured mass% divided by oxides molecular weights
    
df_mol = df_data.div(molecular_weights, axis=1)
df_mol

,Al2O3,Ta2O5,TiO2,MnO,MgO,FeO,SnO2,CaO,Nb2O5
0,0.005280,0.016663,0.010980,0.168205,0.0,0.117823,0.000627,0.002824,0.267146
1,0.000322,0.066127,0.016113,0.113773,0.0,0.137699,0.003548,0.000492,0.193525
2,0.000185,0.058721,0.019107,0.141435,0.0,0.117465,0.003535,0.000374,0.203481
3,0.000550,0.043440,0.019583,0.137197,0.0,0.127672,0.003263,0.001357,0.222693
4,0.000456,0.038272,0.024427,0.136527,0.0,0.133939,0.003368,0.000397,0.224631
...,...,...,...,...,...,...,...,...,...
87,0.006905,0.014506,1.051580,0.000533,0.0,0.049689,0.003807,NaN,0.019638
88,0.006042,0.010229,1.149454,0.000128,0.0,0.031874,0.000318,NaN,0.011775
89,0.023636,0.020865,0.915909,0.000341,0.0,0.062773,0.007075,NaN,0.027162
90,0.002856,0.014936,1.014659,0.000510,0.0,0.057484,0.009291,NaN,0.030172


## 2. Multiply mol amount by amount of oxygen in the oxide
   #### mol amount * oxygen in oxide = oxygen number
       Ex.: molar amount in our case = 0.878765; SiO2 has 2 oxygens in the oxide
       0.878765 * 2 = 1.757531

In [9]:
    # --- create a dataframe with the oxygen numbers

df_oxygen_N = df_mol.mul(oxygen_in_ox,axis=1)
df_oxygen_N

,Al2O3,Ta2O5,TiO2,MnO,MgO,FeO,SnO2,CaO,Nb2O5
0,0.015839,0.083315,0.021959,0.168205,0.0,0.117823,0.001254,0.002824,1.335730
1,0.000967,0.330634,0.032227,0.113773,0.0,0.137699,0.007096,0.000492,0.967624
2,0.000556,0.293605,0.038214,0.141435,0.0,0.117465,0.007071,0.000374,1.017404
3,0.001651,0.217199,0.039166,0.137197,0.0,0.127672,0.006526,0.001357,1.113467
4,0.001368,0.191361,0.048854,0.136527,0.0,0.133939,0.006737,0.000397,1.123154
...,...,...,...,...,...,...,...,...,...
87,0.020714,0.072529,2.103160,0.000533,0.0,0.049689,0.007614,NaN,0.098191
88,0.018125,0.051144,2.298908,0.000128,0.0,0.031874,0.000636,NaN,0.058877
89,0.070909,0.104324,1.831817,0.000341,0.0,0.062773,0.014150,NaN,0.135811
90,0.008568,0.074679,2.029317,0.000510,0.0,0.057484,0.018581,NaN,0.150860


## 3. Sum up the calculated oxygen number for all oxides
An exception here is the occurrence of F or Cl in the analyses. As these commonly occur as anions, they should be subtracted from the total sum. Both elements have -1 charge, which is half the charge of oxygen anions. Consequently, their calculated oxygen number should be multiplied by 0.5 before subtraction of the total.
   #### Σ oxygen numbers of all cations - Σ (0.5 * oxygen numbers of F and Cl) = total oxygen
       Ex.: oxygen number for SiO2 in our case = 1.757531; 
       summing up with other oxides is = 4.58;
       oxygen number for F and Cl is = 0.30 and 0.002 respectively 	
       4.58 - [(0.5 * 0.3)+(0.5 * 0.002)] = 4.429

In [10]:
    # --- add sum of anionic proportions to the last colum in dataframe
    
total_oxygen = df_oxygen_N.sum(axis=1)

    # --- correct total oxygen sum by removing the influence of other anions (F,Cl,etc.)
    
#df_oxygen_N["Sum2"] = df_oxygen_N["SumO"]-(0.5*df_oxygen_N["Cl%"]+0.5*df_data_anion["F%"])

total_oxygen

0     1.746949
1     1.590511
2     1.616125
3     1.644235
4     1.642336
        ...   
87    2.352430
88    2.459691
89    2.220126
90    2.339999
91    2.298904
Length: 92, dtype: float64

## 4. Divide the number of oxygens in the mineral formula unit by total oxygen of last step
The oxygen in the formula unit can be retrieved from the ideal mineral formula. For anhydrous minerals, this value is directly the number of oxygens listed (e.g. 4 in the case of olivine, (Fe,Mg)SiO4). For hydrated phases each other anion (OH, F, or Cl) will count as half oxygen and the final value will be the number of oxygens plus half the number of OH,F,Cl in the formula (e.g. 11 for biotite K(Mg,Fe)3AlSi3O10(OH,F)2)
   #### O in formula unit / total oxygen = oxygen normalization factor 
       Ex.: total oxygen in our case = 4.429; biotite has an equivalent value of 11 oxygens in its formula (considering also the amount of OH and F in the ideal formula - see paragraph above for further clarification)
       11 / 4.429 = 2.484

In [11]:
    # --- create Series that will receive the values for oxygen in formula

O_in_formula = df_analysis['mineral']
#O_in_formula = 2

    # --- assign values of oxygen according to each mineral formula

O_in_formula = np.where(O_in_formula == 'rutile', 2, O_in_formula)
O_in_formula = np.where(O_in_formula == 'coltan', 6, O_in_formula)        
    
    
    # --- amount of oxygen in the formula unit - basis for formula calculation

O_in_formula

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2], dtype=object)

In [12]:
    # --- Oxygen Normalization Factor = number of oxygen in the formula divided by sum of anions
    
ONF = (O_in_formula/total_oxygen)
ONF

0      3.43456
1      3.77237
2      3.71258
3      3.64911
4      3.65333
        ...   
87    0.850185
88     0.81311
89     0.90085
90    0.854701
91     0.86998
Length: 92, dtype: object

## 5. Multiply the oxygen number from item 2 by the oxygen normalization factor 
   #### oxygen number * ONF = anionic proportion
       Ex.: oxygen for SiO2 in our case = 1.757531; ONF = 2.484
       1.757531 * 2.484 = 4.365

In [13]:
    # --- Anionic proportion = oxygen number multiplied by ONF
df_anionic = df_oxygen_N.mul(ONF,axis=0)
df_anionic

,Al2O3,Ta2O5,TiO2,MnO,MgO,FeO,SnO2,CaO,Nb2O5
0,0.0543997,0.286149,0.0754199,0.577711,0,0.404671,0.00430586,0.00970049,4.58764
1,0.00364919,1.24727,0.121571,0.429194,0,0.519452,0.026768,0.0018542,3.65024
2,0.002064,1.09003,0.141874,0.52509,0,0.436099,0.0262499,0.00138945,3.7772
3,0.00602527,0.792582,0.142923,0.500646,0,0.465888,0.023815,0.00495294,4.06317
4,0.0049964,0.699106,0.178479,0.49878,0,0.489323,0.0246121,0.00144931,4.10325
...,...,...,...,...,...,...,...,...,...
87,0.0176105,0.061663,1.78807,0.000453033,0,0.0422451,0.00647362,NaN,0.0834801
88,0.0147372,0.0415854,1.86927,0.000104308,0,0.0259167,0.000516933,NaN,0.0478732
89,0.0638786,0.0939802,1.65019,0.000307321,0,0.0565489,0.0127467,NaN,0.122346
90,0.00732304,0.063828,1.73446,0.000436162,0,0.0491314,0.0158815,NaN,0.12894


## 6. Multiply the anionic proportion by the ratio of cations and oxygen in the oxide 
   #### anionic proportion * (cations in oxide / oxygen in oxide) = cationic proportion
       Ex.: in our case anionic proportion = 4.365; and the ratio between Si and O in the oxide formula (i.e. SiO2) is 1/2 = 0.5
       4.365 * 0.5 = 2.183

In [15]:
    # --- Cationic proportion = anionic proportion multiplied by oxide cation ratio
df_cationic = df_anionic.mul(cation_ratio_ox)

    # --- rename dataframe columns for simplification


df_cationic.columns = ['Al','Ta','Ti','Mn','Mg','Fe','Sn','Ca','Nb']

    # --- combine calculated data with metadata
#df_cationic['Total'] = df_cationic.sum(axis=0,numeric_only=True)

df_cationic.insert(0,"Sample",df_analysis["Sample"])
df_cationic.insert(1,"Area",df_analysis["Area"])
df_cationic.insert(2,"Comment",df_analysis["Comment"])
df_cationic.insert(3,"Mineral",df_analysis["mineral"])


df_cationic.to_csv('Ore_WDS_apfu.csv', index=False)
df_cationic

,Sample,Area,Comment,Mineral,Al,Ta,Ti,Mn,Mg,Fe,Sn,Ca,Nb
0,BU04,Ore1,BU04-Ore1.2,coltan,0.0362664,0.11446,0.03771,0.577711,0,0.404671,0.00215293,0.00970049,1.83506
1,BU04,Ore1,BU04-Ore1.3,coltan,0.00243279,0.498909,0.0607857,0.429194,0,0.519452,0.013384,0.0018542,1.4601
2,BU04,Ore1,BU04-Ore1.4,coltan,0.001376,0.436013,0.0709372,0.52509,0,0.436099,0.013125,0.00138945,1.51088
3,BU04,Ore1,BU04-Ore1.5,coltan,0.00401685,0.317033,0.0714615,0.500646,0,0.465888,0.0119075,0.00495294,1.62527
4,BU04,Ore1,BU04-Ore1.6,coltan,0.00333093,0.279643,0.0892394,0.49878,0,0.489323,0.012306,0.00144931,1.6413
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,BU19w,Ore4,BU19w-Rut4.1,rutile,0.0117403,0.0246652,0.894037,0.000453033,0,0.0422451,0.00323681,NaN,0.033392
88,BU19w,Ore4,BU19w-Rut4.2,rutile,0.00982483,0.0166342,0.934633,0.000104308,0,0.0259167,0.000258466,NaN,0.0191493
89,BU19w,Ore4,BU19w-Rut4.3,rutile,0.0425857,0.0375921,0.825096,0.000307321,0,0.0565489,0.00637337,NaN,0.0489383
90,BU19w,Ore5,BU19w-Area7_ore.2,rutile,0.00488203,0.0255312,0.86723,0.000436162,0,0.0491314,0.00794075,NaN,0.051576


In [ ]:
df_data_cationic["Li"] = (number_oxygen/4) - (df_data_cationic["K"]+df_data_cationic["Na"]+df_data_cationic["Ca"])
df_data_cationic["OH"] = (number_oxygen/4) - (df_data_cationic["Cl"]+df_data_cationic["F"])
df_data_cationic